# references
1. https://www.kaggle.com/columbia2131/device-eda-interpolate-by-removing-device-en-ja

# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
notebookName = 'Baseline'
PATH = Path(f"./models/{notebookName}")

In [4]:
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0

0
1
2
3


In [5]:
num_files

4

In [6]:
RANDOM_STATE = 1990
lr = 0.0005
batch_size = 128
EPOCH_NUM = 1000

torch.manual_seed(RANDOM_STATE)

experience_name = f"{num_files}"
checkpoint_name = "check_point"
model_name = str("model - " + experience_name)
param_name = str("param - " + experience_name)
result_name = str("result - " + experience_name)

dummy_path = str(PATH / f"{num_files}")
checkpoint_path = str(PATH / f"{checkpoint_name}.pth")
model_path = str(PATH / f"{model_name}.pth")
param_path = str(PATH / f"{param_name}.pth")
result_path = str(PATH / f"{result_name}.csv")
model_path, param_path, result_path

('models\\Baseline\\model - 4.pth',
 'models\\Baseline\\param - 4.pth',
 'models\\Baseline\\result - 4.csv')

# Useful Functions

In [7]:
def calc_haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    
    c = 2 * np.arcsin(a ** 0.5)
    dist = 6_367_000 * c
    return dist

In [8]:
def check_score(input_df: pd.DataFrame) -> pd.DataFrame:
    output_df = input_df.copy()
    
    output_df['meter'] = input_df.apply(
        lambda r: calc_haversine(
            r.latDeg, r.lngDeg, r.t_latDeg, r.t_lngDeg
        ),
        axis=1
    )

    meter_score = output_df['meter'].mean()
    print(f'error meter: {meter_score}')

    scores = []
    for phone in output_df['phone'].unique():
        _index = output_df['phone']==phone
        p_50 = np.percentile(output_df.loc[_index, 'meter'], 50)
        p_95 = np.percentile(output_df.loc[_index, 'meter'], 95)
        scores.append(p_50)
        scores.append(p_95)

    score = sum(scores) / len(scores)
    print(f'score: {score}')
    
    return output_df, meter_score , score

In [9]:
def check_score_np(predict:torch.Tensor, target:torch.Tensor):
    m = []
    predict = predict.detach().numpy()
    target = target.detach().numpy()
    for i in range(predict.shape[0]):
        temp = calc_haversine(predict[i,0], predict[i,1], target[i,0], target[i,1])
        m.append(temp)
    
    m = np.array(m)
    score = (np.percentile(m, 50) + np.percentile(m, 95))/2
    
    return score

# Load Datasets

In [10]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")
df_train = pd.read_pickle(str(data_dir / "gsdc_train.pkl.gzip"))

In [ ]:
# check score
df_train, default_loss, default_meas = check_score(df_train)

# Feature Engineering
## Simple view, what is in data frame.

In [ ]:
print(df_train.shape)
df_train.head()

In [ ]:
for c in df_train.columns:
    print(c)
    print(df_train[c].describe())
    print()

In [ ]:
for col in df_train.columns:
    print(col)

In [ ]:
df_train[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM',
             'xSatVelMps', 'ySatVelMps', 'zSatVelMps', 
             'xSatPosM', 'ySatPosM', 'zSatPosM',
            'UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec', 
            'DriftXRadPerSec', 'DriftYRadPerSec', 'DriftZRadPerSec']].info()

In [ ]:
# ref: https://towardsdatascience.com/fast-and-robust-sliding-window-vectorization-with-numpy-3ad950ed62f5
def extract_window(src:np.array, sequence_length = 16)->np.array:
    output = []
    for i in range(src.shape[0]):
        index = np.arange(i-sequence_length, i, 1)
        index[index < 0] = 0
        
        output.append(np.expand_dims(src[index,:], 0))
            
    output = np.concatenate(output, axis = 0)
    return output
    

def extract_features(df, sequence_length = 16, train = True):
    Xcols = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM',
             'xSatVelMps', 'ySatVelMps', 'zSatVelMps', 
             'xSatPosM', 'ySatPosM', 'zSatPosM',
            'UncalGyroXRadPerSec', 'UncalGyroYRadPerSec', 'UncalGyroZRadPerSec', 
            'DriftXRadPerSec', 'DriftYRadPerSec', 'DriftZRadPerSec']
    Ycols = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM', 'speedMps', 'courseDegree']
    
    X = df[Xcols]
    if train:
        y = df[Ycols]
    else:
        y = None
        
    X = extract_window(np.array(X), sequence_length = sequence_length)
    y = np.array(y)
    
    return X, y

X, y = extract_features(df_train)

print(X.shape, y.shape)

# Modeling

In [ ]:
# build model
class SimpleNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 1024)
        self.fc4 = nn.Linear(1024, 1024)
        self.fc5 = nn.Linear(1024, output_size)
        
        self.dropout_30 = nn.Dropout(p = 0.3)
        self.dropout_60 = nn.Dropout(p = 0.6)
    
    def forward(self, x):
        x = torch.flatten(x, start_dim = 1)
        
        x =  F.relu(self.fc1(x))
        x = self.dropout_60(x)
        x =  F.relu(self.fc2(x))
        x = self.dropout_60(x)
        x =  F.relu(self.fc3(x))
        x = self.dropout_30(x)
        x =  F.relu(self.fc4(x))
        x = self.dropout_30(x)
        x = self.fc5(x)
        
        return x
    
model = SimpleNetwork(X.shape[1] * X.shape[2], y.shape[1])
model.to(device)
loss_func = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

In [ ]:
# Create DataLoader
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/7, random_state = 1990)

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)

loader_train = DataLoader(ds_train, batch_size = batch_size, shuffle = True)
loader_test = DataLoader(ds_test, batch_size = batch_size, shuffle = False)

In [ ]:
X

In [ ]:
def train(epoch):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    for data, targets in loader_train:
        data = data.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs)
        ground.append(targets)

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = check_score_np(predict.to('cpu'), ground.to('cpu'))
    return loss, meas

In [ ]:
def test():
    model.eval()  # 신경망을 추론 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets in loader_test:
            data = data.to(device)
            targets = targets.to(device)

            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            predict.append(outputs)
            ground.append(targets)

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = check_score_np(predict.to('cpu'), ground.to('cpu'))
    return loss, meas

In [ ]:
test()

In [ ]:
history = []

check_meas = np.inf
check_loss = np.inf
check_epoch = 0

for epoch in range(EPOCH_NUM):
    train_loss, train_meas = train(epoch)
    test_loss, test_meas = test()
    
    history.append({'epoch':epoch, 'train_loss':train_loss, 'train_meas':train_meas, 'test_loss':test_loss, 'test_meas':test_meas})
    
    if (test_meas < check_meas):
        print("")
        print(f"/***CHECK_POINT***/ ")
        print(f"TRAIN - {train_loss}, {train_meas}")
        print(f"TEST - {test_loss}, {test_meas}")
        print("")
        check_meas = test_meas
        check_loss = test_loss
        check_epoch = epoch
        torch.save(model.state_dict(), checkpoint_path)
    
    print(f"/*** EPOCH : {epoch+1}/{EPOCH_NUM} ***/")
    print(f"TRAIN - {train_loss}, {train_meas}")
    print(f"TEST - {test_loss}, {test_meas}")
    print("")
    
df_history = pd.DataFrame(history)

In [ ]:
df_history = pd.DataFrame(history)

fig, axes = plt.subplots(2,1,figsize = (8,12))

axes[0].plot(df_history['epoch'], df_history['train_loss'])
axes[0].plot(df_history['epoch'], df_history['test_loss'])
axes[0].axvline(x = check_epoch, ymin = 0, ymax = df_history['test_loss'].max(), color = 'r')
axes[0].axhline(y = default_loss, xmin = 0, xmax = df_history['epoch'].max(), color = 'k')

axes[1].plot(df_history['epoch'], df_history['train_meas'])
axes[1].plot(df_history['epoch'], df_history['test_meas'])
axes[1].axvline(x = check_epoch, ymin = 0, ymax = df_history['test_meas'].max(), color = 'r')
axes[1].axhline(y = default_meas, xmin = 0, xmax = df_history['epoch'].max(), color = 'k')

In [ ]:
del X_train, X_test, y_train, y_test

# Submission

In [ ]:
df_test = pd.read_pickle(str(data_dir / "gsdc_test.pkl.gzip"))

print(df_test.shape)
df_test.head()

In [ ]:
model.load_state_dict(torch.load(checkpoint_path))
torch.save(model.state_dict(), model_path)

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)

In [ ]:
model = SimpleNetwork(X.shape[1] * X.shape[2], y.shape[1])
model = model.to(device)
model.load_state_dict(torch.load(model_path))

In [ ]:
X, _ = extract_features(df_test, train = False)

X = torch.Tensor(X)

loader_test = DataLoader(X, batch_size = batch_size, shuffle = False)

In [ ]:
model.eval()  # 신경망을 추론 모드로 전환

# 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
predict = []
with torch.no_grad():  # 추론 과정에는 미분이 필요없음
    for data in loader_test:
        data = data.to(device)

        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        predict.append(outputs)
predict = torch.cat(predict, axis = 0).to('cpu')

In [ ]:
predict.shape

In [ ]:
print(submission.shape)
submission.columns

In [ ]:
submission['latDeg'] = predict[:,1]
submission['lngDeg'] = predict[:,2]

In [ ]:
submission.to_csv(f"./models/{notebookName}/{num_files} - result.csv", index = False)
pd.DataFrame([]).to_csv(dummy_path)